In [1]:
from pytorch_libraries.utility_dataset import *
from pytorch_libraries.model_DNN import *

from IPython.display import clear_output

from pylab import *
%matplotlib inline

# 0. Load dataset and Model

In [11]:
trnset = np.fromfile("../data/MNIST/MNIST_train_data.dat").reshape(-1,785)
tstset = np.fromfile("../data/MNIST/MNIST_test_data.dat" ).reshape(-1,785)
trnset[:,-1] = trnset[:,-1] -1
tstset[:,-1] = tstset[:,-1] -1

model = DNN_4l(784,256,64,32,10).cuda()
#torch.load('../data/model/MNIST_DNN.pt').cuda()

trnset = tcDataset(trnset,shape=(-1,784))
tstset = tcDataset(tstset,shape=(-1,784))

# 1. Training

In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
batches   = DataLoader(trnset,batch_size=100, shuffle=True)

model.train()
# begin Training
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(batches):
        
        # 1.get the inputs
        inputs = Variable(batch["data"]).cuda()
        labels = Variable(batch["label"]).cuda()
        # 2.zero the parameter gradients
        optimizer.zero_grad()
        # 3.forward + backward + optimize
        outputs = model(inputs)
        loss    = F.nll_loss( outputs, labels )
        loss.backward()
        optimizer.step()
        
        
        # TRAINING DESHBOARD #
        ##################################################################
        running_loss += loss.data[0]
        if i % 100 == 99:    # print every 100 mini-batches
            predict = (outputs.data).max(1)[1]
            correct = predict.eq(labels.data).sum()
            accuracy= correct/batches.batch_size
            print('[{:5d}, {:5d}] loss: {:5.3f}, acc: {:5.3f}'.
                  format(epoch+1, i+1, running_loss/100, accuracy))
            clear_output(wait=True)
            running_loss = 0.0
        ##################################################################


[   10,   600] loss: 0.067, acc: 0.950


# 2. Test Accuracy

In [13]:
corrects = 0
model.eval()
for item in DataLoader(tstset):
    inputs   = Variable(item["data"]).cuda()
    labels   = Variable(item["label"]).cuda()
    outputs  = model(inputs)
    predicts = (outputs.data).max(1)[1] # get the index of the max log-probability
    corrects = corrects + predicts.eq(labels.data).sum()
    
print("acc:{:5.3f}".format(corrects/len(tstset)))

acc:0.979


In [14]:
torch.save(model.cpu(),'../data/model/MNIST_DNN.pt') 